[<img src="https://drive.google.com/uc?export=view&id=1D6PD5jhrfNa_D7yfn7ha7OjJslVPy8qg" width="500" align="top" />](https://drive.google.com/uc?export=view&id=1D6PD5jhrfNa_D7yfn7ha7OjJslVPy8qg)

#**ColabPCR**
Polymerase Chain Reaction (PCR) amplification of specific genomic regions within eukaryotic genomes frequently exhibits a diminished success rate, often attributed to the non-specific annealing of primers at multiple genomic loci. To mitigate this challenge, we developed ***ColabPCR***, a program meticulously designed for the optimal selection and evaluation of primers targeting a specified genomic region. ColabPCR refines primer length and melting temperature parameters through the utilization of Primer3 software, and it quantifies the number of potential binding regions within the target genome via blastn analysis. Furthermore, the program facilitates the integration of restriction enzyme recognition sites at the 5' termini of primers and provides a mechanism to confirm the absence of such sites within the intended amplification region.


###**Input**:  
* ***Target Genome***:
  * For _Dacus carota_, DCARv2 and DH1v3 genome accessions can be selected from the drop down menu
  * Genomes can be uploaded to Google Colab
  * Genomes can be downloaded from NCBI Assembly by accession number
  * A local file can be specified

* Primers are designed using primer3 defaults

* Restriction enzymes can be specifed. The selected RE will be added to the 5'end of primers and if they are within the target region available RE will be displayed

* positions of the region to amplify: chromosome index `int`, gene start `int`, number of bases from the gene start or end




#***Install software and download the selected genome***


In [ ]:
#@title ##Install software and load functions
import os,sys,json,glob,requests,shutil

#@markdown ###Install programs
if shutil.which('blastn') is None:
  print('Installing BLAST...')
  !curl -l ftp://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ > ncbi_blast_files.txt 2> /dev/null
  blast_file_name = ''
  with open('ncbi_blast_files.txt','r') as f:
    for line in f:
      if ('+-x64-linux.tar.gz' in line) and (not '.md5' in line):
        blast_file_name = line.strip()
  if len(blast_file_name) > 0:
    url = 'https://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/'+blast_file_name
    if not os.path.exists(blast_file_name):
      !wget {url} 2> /dev/null 1> logs.txt
      !tar -xzvf {blast_file_name} 2> /dev/null 1> logs.txt
      blast_folder = blast_file_name.split('+')[0]+'+/bin'
    else:
      blast_folder = blast_file_name.split('+')[0]+'+/bin'
      print('Blast already installed...')
  else:
    print('Error downloading Blast')
  os.environ['PATH'] += f':/content/{blast_folder}'
else:
  print('Blast already installed...')

if shutil.which('primer3_core') is None:
  print('Installing Primer3')
  !sudo apt-get install -y build-essential g++ cmake git-all -qq 2> /dev/null 1>> logs.txt
  !git clone https://github.com/primer3-org/primer3.git primer3
  %cd primer3/src
  !make all 2> /dev/null 1>> logs.txt
  #!make test 2> /dev/null 1>> logs.txt
  os.environ['PATH'] += f':/content/primer3/src'
  %cd /content
else:
  print('Primer3 already installed...')

if not 'Bio' in sys.modules:
  print('Installing Biopython')
  !pip install Bio 2> /dev/null 1> logs.txt
else:
  print('Biopython already installed...')

from Bio import Restriction
from Bio import SeqIO

if not os.path.exists('./datasets'):
  print('Installing NCBI Datasets')
  !wget https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/v2/linux-amd64/datasets 2> /dev/null 1>> logs.txt
  !chmod +x datasets
else:
  print('NCBI Datasets already installed...')

#@markdown ###Load functions
print('Loading functions...')
print('Region extraction function...')

def extractRegion(mode,cIndex,Product_Size, start=0,end=0,gpstart = 0,gpend = 0,ntreg=10000, locus_tag='', flexibility=200, fixedNear = False, fixedFar=False):
  strand = 1
  found = False
  for r in SeqIO.parse(gfile,'genbank'):
    if r.id == chromosomes[cIndex][0]:
      if mode == 'region':
        start = start-1
        end = end
        Product_Size = end-start
        print(f'Target location: Start:{start+1} - End:{end}\n')
      elif mode == 'gene':
        for f in r.features:
          if f.type == 'gene':
            if ('locus_tag' in f.qualifiers):
              if f.qualifiers["locus_tag"][0] == locus_tag:
                  lstart = f.location.start
                  lend = f.location.end
                  strand = f.location.strand
                  if gpend == 0:
                    gpend = abs(lend-lstart)
                  if strand == 1:
                    start = lstart+gpstart
                    end = lstart+gpend
                  else:
                    end = lend-gpstart
                    start = lend-gpend
                  Product_Size = abs(lend-lstart)
                  print(f'Target location: Start:{start+1} - End:{end}\nLocus tag location: Start:{lstart+1} - End:{lend} strand {strand}')
      else:
        for f in r.features:
          if f.type == 'CDS':
            if ('locus_tag' in f.qualifiers):
              if f.qualifiers["locus_tag"][0] == locus_tag:
                if mode == 'promoter':
                  found = True
                  lstart = f.location.start
                  lend = f.location.end
                  strand = f.location.strand
                  if strand == 1:
                    start = lstart-Product_Size
                    end = lstart
                  else:
                    start = lend
                    end = lend+Product_Size
                  print(f'Target location: Start:{start+1} - End:{end}\nLocus tag location: Start:{lstart+1} - End:{lend} strand {strand}')
                elif mode == 'terminator':
                  found = True
                  lstart = f.location.start
                  lend = f.location.end
                  strand = f.location.strand
                  if strand == 1:
                    start = lend
                    end = lend+Product_Size
                  else:
                    start = lstart-Product_Size
                    end = lstart
                  print(f'Target location: Start:{start+1} - End: {end}\nLocus tag location: Start:{lstart+1} - End:{lend} strand {strand}')
        ###
        if not found:
          raise Exception(f'{locus_tag} not found.')
      ###
      targetSeq = r[start:end]
      #Checks for overlapping CDSs
      print(f'The following features are in the +- {ntreg} nucleotides range:')
      ltags = []
      for f in r[start-ntreg:end+ntreg].features:
        if ('locus_tag' in f.qualifiers):
          #if f.type == 'CDS':
          if not f.qualifiers["locus_tag"][0] == locus_tag:
            if not f.qualifiers["locus_tag"][0] in ltags:
              ltags.append(f.qualifiers["locus_tag"][0])
              print(f'--> {f.qualifiers["locus_tag"][0]}-{start+1-ntreg+f.location.start}-{start-ntreg+f.location.end}')
              feat_len = abs(f.location.end - f.location.start)
              feat_start = min(f.location.end,f.location.start)
              feat_end = max(f.location.end,f.location.start)
              if (feat_start in range(ntreg,ntreg+Product_Size)) or (feat_end -1 in range(ntreg,ntreg+Product_Size)):
                print(f'  * This feature overlaps the selected promoter region... Start: {start+1-ntreg+feat_start} - End:{start-ntreg+feat_end} Range: {start+1}-{start+Product_Size}')
      ###
      if flexibility > Product_Size/2:
        flexibility = int(Product_Size/3)
        print(f'The program will allow to move the region bounds {flexibility} nucleotides on each side')
      ###default return values
      inStart = start
      inEnd = end
      FORCE_RIGHT_START = False
      FORCE_LEFT_START = False
      ###
      if  mode == 'region':
        if fixedNear and not fixedFar and (strand == 1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 - flexibility
          inStart = 0
          FORCE_LEFT_START = True
        elif fixedFar and not fixedNear and (strand == 1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1
          FORCE_RIGHT_START = True
        elif fixedFar and fixedNear:
          print(f'Fixing the primer on the both sides. Start: {start+1} End: {end} Strand: {strand}')
          inStart = 0
          inEnd = end - start - 1
          FORCE_RIGHT_START = True
          FORCE_LEFT_START = True
        else:
          print(f'Both target ends set to flexible ({flexibility}). Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1 - flexibility
      elif  mode == 'gene':
        if fixedNear and not fixedFar and (strand == 1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 - flexibility
          inStart = 0
          FORCE_LEFT_START = True
        elif fixedNear and not fixedFar and (strand == -1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1
          inStart = flexibility
          FORCE_RIGHT_START = True
        elif fixedFar and not fixedNear and (strand == 1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1
          FORCE_RIGHT_START = True
        elif fixedFar and not fixedNear and (strand == -1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 - flexibility
          inStart = 0
          FORCE_LEFT_START = True
        elif fixedFar and fixedNear:
          print(f'Fixing the primer on the both sides. Start: {start+1} End: {end} Strand: {strand}')
          inStart = 0
          inEnd = end - start - 1
          FORCE_RIGHT_START = True
          FORCE_LEFT_START = True
        else:
          print(f'Both target ends set to flexible ({flexibility}). Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1 - flexibility
      ###
      elif  mode == 'promoter':
        if fixedNear and not fixedFar and (strand == 1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1
          inStart = flexibility
          FORCE_RIGHT_START = True
        elif fixedNear and not fixedFar and (strand == -1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 - flexibility
          inStart = 0
          FORCE_LEFT_START = True
        elif fixedFar and not fixedNear and (strand == 1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inStart = 0
          inEnd = end - start - 1 - flexibility
          FORCE_LEFT_START = True
        elif fixedFar and not fixedNear and (strand == -1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1
          inStart = flexibility
          FORCE_RIGHT_START = True
        elif fixedFar and fixedNear:
          print(f'Fixing the primer on the both sides. Start: {start+1} End: {end} Strand: {strand}')
          inStart = 0
          inEnd = end - start - 1
          FORCE_RIGHT_START = True
          FORCE_LEFT_START = True
        else:
          print(f'Both target ends set to flexible ({flexibility}). Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1 - flexibility
      ###

      elif  mode == 'terminator':
        if fixedNear and not fixedFar and (strand == 1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 - flexibility
          inStart = 0
          FORCE_LEFT_START = True
        elif fixedNear and not fixedFar and (strand == -1):
          print(f'Fixing the primer on the start codon side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 - flexibility
          inStart = flexibility
          FORCE_RIGHT_START = True
        elif fixedFar and not fixedNear and (strand == 1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1
          FORCE_RIGHT_START = True
        elif fixedFar and not fixedNear and (strand == -1):
          print(f'Fixing the primer on the distant side. Start: {start+1} End: {end} Strand: {strand}')
          inEnd = end - start - 1 -flexibility
          inStart = 0
          FORCE_LEFT_START = True
        elif fixedFar and fixedNear:
          FORCE_RIGHT_START = True
          FORCE_LEFT_START = True
          print(f'Fixing the primer on the both sides. Start: {start+1} End: {end} Strand: {strand}')
          inStart = 0
          inEnd = end - start - 1
        else:
          print(f'Both target ends set to flexible ({flexibility}). Start: {start+1} End: {end} Strand: {strand}')
          inStart = flexibility
          inEnd = end - start - 1 - flexibility
      ###
      if (not fixedNear) and (not fixedFar):
        prangemin = end - start - 2*flexibility
      elif (not fixedNear) or (not fixedFar):
        prangemin = end - start - flexibility
      else:
        prangemin = end - start
      prangemax = end - start
      targetSeq = r[start:end]
      SeqIO.write(targetSeq, 'target.fasta','fasta')
      print(f'Target product size (target.fasta): {len(targetSeq)}')
      print(f'Flexible Product size range: {prangemin}-{prangemax}')
      print(f'{targetSeq.seq}')
      return(targetSeq,inStart,inEnd,prangemin,prangemax,FORCE_RIGHT_START,FORCE_LEFT_START)
  print(f'Error extracting region.')

##virtual PCR functions
print('Virtual PCR functions...')

import pandas as pd

def PCR(q,oligo_5_prima_Fwd,oligo_5_prima_rv,gfile,max_diff_aln_len,remove_oligos):
  qids = []
  dbname = os.path.splitext(os.path.basename(gfile))[0]
  if remove_oligos:
    for r in SeqIO.parse(q,'fasta'):
      if str(r.seq).startswith(oligo_5_prima_rv):
        r.seq = r.seq[len(oligo_5_prima_rv):]
      elif str(r.seq).startswith(oligo_5_prima_Fwd):
        r.seq = r.seq[len(oligo_5_prima_Fwd):]
      qids.append(r)
    SeqIO.write(qids,'query.fasta','fasta')
    query = 'query.fasta'
  else:
    query = q
    max_diff_aln_len += max(len(oligo_5_prima_rv),len(oligo_5_prima_Fwd))

  #Blast
  cpu_count = os.cpu_count()
  if cpu_count > 1:
    if not os.path.exists(f'{dbname}.nsq'):
      fnaFile = os.path.splitext(gfile)[0]+'.fna'
      SeqIO.convert(gfile,'gb',fnaFile,'fasta')
      os.system(f'makeblastdb -in {fnaFile} -dbtype nucl -out {dbname}')
    else:
      print(f'Using existent blast database: {dbname} ')
    os.system(f'blastn -query {query} -db {dbname} -task blastn-short -word_size 4 -evalue 1 -outfmt "6 qseqid sseqid length qlen qstart qend slen sstart send sstrand qseq sseq evalue pident mismatch qcovs qcovhsp qcovus" -max_target_seqs 10 -out blast.res -num_threads {cpu_count}')
  else:
    if not os.path.exists(fnaFile):
      fnaFile = os.path.splitext(gfile)[0]+'.fna'
      SeqIO.convert(gfile,'gb',fnaFile,'fasta')
    os.system(f'blastn -query {query} -subject {fnaFile} -task blastn-short -word_size 4 -evalue 1 -outfmt "6 qseqid sseqid length qlen qstart qend slen sstart send sstrand qseq sseq evalue pident mismatch qcovs qcovhsp qcovus" -max_target_seqs 10 -out blast.res')
  blast = pd.read_csv('blast.res',sep='\t',names=['qseqid','sseqid','length','qlen','qstart','qend','slen','sstart','send','sstrand', 'qseq','sseq','evalue','pident','mismatch','qcovs','qcovhsp','qcovus'], index_col=False)
  blast = blast[blast['length'] >= blast['qlen']-max_diff_aln_len]
  return(blast)

def primer_blast(q,oligo_5_prima_Fwd,oligo_5_prima_rv,gfile,primerPair,max_diff_aln_len,remove_oligos):
  #correr blast para encontrar hits de primers en genoma
  blast = PCR(q,oligo_5_prima_Fwd,oligo_5_prima_rv,gfile,max_diff_aln_len,remove_oligos)
  blast.to_csv(f'FilteredPrimerHits_{primerPair}.csv')

  #
  qseqids = blast['qseqid'].unique()
  if len(qseqids) != 2:
    sys.exit(f'Error: Either the query has more than two primers, or only one primer was found on the target genome...')
  else:
    fwd = qseqids[0]
    rv = qseqids[1]
  #new 7/11/2024 Counts number of hits
  fwd_found_no_mismatch = len(blast[(blast['qseqid'] == fwd) & (blast['mismatch'] == 0)])
  rv_found_no_mismatch = len(blast[(blast['qseqid'] == rv) & (blast['mismatch'] == 0)])
  fwd_found_with_mismatch = len(blast[(blast['qseqid'] == fwd) & (blast['mismatch'] > 0)])
  rv_found_with_mismatch = len(blast[(blast['qseqid'] == rv) & (blast['mismatch'] > 0)])
  #end_new
  #analizar tabla de blast y buscar resultados compatibles con productos de amplificación.
  with open('virtualPCR.csv','a+') as f:
    #new 7/11/2024_Table headers
    #end_new
    numProducts = 0
    sseqids = blast['sseqid'].unique()
    for sid in sseqids:
      sblast = blast[blast['sseqid'] == sid]
      if len(sblast) >= 2:
        sfwdblast = sblast[sblast['qseqid']==fwd]
        srvblast = sblast[sblast['qseqid']==rv]
        if (len(sfwdblast) > 0 ) and (len(srvblast) > 0):
          for _,fwdHit in sfwdblast.iterrows():
            if fwdHit['sstrand'] == 'plus':
              rvStrand = 'minus'
              start = fwdHit['sstart']
            elif fwdHit['sstrand'] == 'minus':
              rvStrand = 'plus'
              end = fwdHit['sstart']
            srvblastMinus = srvblast[srvblast['sstrand'] == rvStrand]
            if len(srvblastMinus) > 0:
              for _,rvHit in srvblastMinus.iterrows():
                if rvHit['sstrand'] == 'minus':
                  if rvHit['send'] > fwdHit['send']:
                    end = rvHit['sstart']
                    pcrProducLen = end - start + 1
                  else:
                    end = rvHit['sstart']
                    pcrProducLen = 0
                elif rvHit['sstrand'] == 'plus':
                  if rvHit['send'] < fwdHit['send']:
                    start = rvHit['sstart']
                    pcrProducLen = end - start + 1
                  else:
                    start = rvHit['sstart']
                    pcrProducLen = 0
                #Primer Fwd,Primer Fwd length,Fwd Aln Length, Fwd Hit QSeq, Fwd Hit SSeq,
                #Fwd Percent ID, Fwd Mismatches, Primer Rv, Primer Rv length,
                #Rv Aln Length, Rv Hit QSeq, Rv Hit SSeq, Rv Percent ID, Rv Mismatches, Product Length'
                #new_updated_7/11/2024 to add fwd and rv hjit counts
                f.write(f"{fwdHit['sseqid']},{start},{end},{fwdHit['qseqid']},{fwdHit['qlen']},{fwd_found_no_mismatch},{fwd_found_with_mismatch},{fwdHit['length']},{fwdHit['qseq']},{fwdHit['sseq']},{fwdHit['pident']},{fwdHit['mismatch']},{rvHit['qseqid']},{rvHit['qlen']},{rv_found_no_mismatch},{rv_found_with_mismatch},{rvHit['length']},{rvHit['qseq']},{rvHit['sseq']},{rvHit['pident']},{rvHit['mismatch']}")
                #end_new
                if pcrProducLen == 0:
                  f.write(f',Divergent primers\n')
                elif pcrProducLen < 20000:
                  f.write(f',{pcrProducLen}\n')
                  numProducts+=1
                else:
                  f.write(f',{pcrProducLen} Long PCR Product\n')

  print(f'Number of PCR products of length < 20000: {numProducts}.')
  return(numProducts)

#validate_oligos
def validate_oligo(oligo):
  if len(oligo) > 0:
    for i in oligo:
      if i not in 'ATCG':
        return(False)
    return(True)
  else:
    return(False)


In [ ]:
#@title ## Download target genome

#@markdown ### Select target genome mode from the dropdown menu. </br>
#@markdown ##### To download the target genome from [NCBI Assembly/Dataset genomes](https://www.ncbi.nlm.nih.gov/datasets/genome/) use: NCBI Assembly Accession number.
 #@markdown * **Accsession number example**: GCA_001625215.2 [*Daucus carota*]
#@markdown ##### To download *Daucus carota* genome select the Genome version from the dropdown menu.
#@markdown ##### To upload a genome in genebank format, select: upload
#@markdown ##### To use a local file, select: local_file
#@markdown <br>
Genome_Mode = 'NCBI Assembly Accession number' #@param ["NCBI Assembly Accession number","upload file","local_file","Daucus carota DH1v3", "Daucus carota DCARv2"]
#@markdown <br>

from Bio import SeqIO
import os

def downloadGenome(acc,gfile):
  if not os.path.exists(gfile):
    !./datasets download genome accession {acc} --include gbff --filename {acc}.zip >/dev/null 2>&1
    !unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/*.gbff -d ./ >/dev/null 2>&1
    os.remove(f'{acc}.zip')
    !mv genomic.gbff {gfile} >/dev/null 2>&1
    pass
  else:
    pass



#@markdown ##### If you selected **"NCBI Assembly Accession number"** please input the accession number in the form.
NCBI_genome_Accession = "" #@param {type:'string'}
#@markdown ##### <br> If you selected **"Local file"** please input the path to the genome in genbank format.
local_file = "" #@param {type:'string'}

if Genome_Mode == 'NCBI Assembly Accession number':
  #
  if len(NCBI_genome_Accession) > 0:
    if 'GCA' in NCBI_genome_Accession or 'GCF' in NCBI_genome_Accession:
      acc = NCBI_genome_Accession
      print(f'Downloading {acc}')
      gfile = f'{acc}.gb'
      downloadGenome(acc,gfile)
    else:
      raise Exception(f'{NCBI_genome_Accession} is not a NCBI assembly accession number.')
elif Genome_Mode == 'Daucus carota DH1v3':
  print('Downloading genome:\n--> DH1v3: GCA_001625215.2')
  acc = 'GCA_001625215.2'
  gfile = 'dh3v1.gb'
  downloadGenome(acc,gfile)
elif Genome_Mode == 'Daucus carota DCARv2':
  print('Downloading genome:\n--> DCARv2: GCA_001625215.1')
  gfile = 'dcarv2.gb'
  acc = 'GCA_001625215.1'
  downloadGenome(acc,gfile)
elif Genome_Mode == 'upload file':
  from google.colab import files
  uploaded = files.upload()
  gfile = uploaded.keys()[0]
  try:
    gfileName = os.path.splitext(gfile)[0]+".gb"
    SeqIO.convert(gfile,'fasta',gfileName,'gb')
    os.remove(gfile)
    gfile = gfileName
    print(f'Uploaded file {gfile} converted to Genbank format.\nWarning: Locus_tag region extraction requires an annotated genome.')
  except:
    print(f'Uploaded file {gfile}')
elif Genome_Mode == 'local_file':
  if os.path.exists(local_file):
    try:
      gfileName = os.path.splitext(local_file)[0]+".gb"
      SeqIO.convert(local_file,'fasta',gfileName,'gb')
      os.remove(local_file)
      local_file = gfileName
      print(f'{local_file} converted to Genbank format.\nWarning: Locus_tag region extraction requires an annotated genome.')
    except:
      print(f'Local file loaded: {local_file}')
  else:
    print(f'Error: {local_file} does not exist.')
else:
  raise Exception(f'Error: the selected genome could not be downloaded.')

#Get the list of chromosomes/scaffolds/contigs
chromosomes = {}

for i,r in enumerate(SeqIO.parse(gfile,'genbank')):
  chromosomes[i] = [r.id,r.description]

for i in chromosomes:
  print(f'Chromosome index [cindex] {i}: {chromosomes[i][0]} - {chromosomes[i][1]}')





---



#**Extract target sequence from the genome**

In [ ]:
#@title #Extract the target sequence from the downloaded genome
#@markdown ### **Requiered data**:
#@markdown - chromosome index form the previous list [0 indexed]
#@markdown - start and end location [for region mode]
#@markdown - locus_tag [for gene and promoter/terminator mode]
#@markdown - start and end positions within a gene [for gene mode]
#@markdown - Product size [for promoter/terminator mode]
#@markdown - primer region end flexibility

#@markdown <br>
import sys
breake_execution = False
flexibility = 0
fixedFar = False
fixedNear = False

#@markdown ###**Select the running mode**
mode = 'region' #@param ['promoter','terminator','region','gene']

#@markdown #### <br/>**Chromosome index** [Required for all running modes]
cIndex = 0 #@param {type:"integer"}
if cIndex == None or not isinstance(cIndex,int):
  breake_execution = True

#@markdown #### <br/>**Locus tag** [Only used in the gene and Promoter/terminator modes]
locus_tag = ''
if mode == 'gene' or mode == 'promoter' or mode == 'terminator':
  locus_tag = ''  #@param {type:"string"}

#@markdown ####<br/> **Start and end genomic coordinates** [Required only in the region mode]
start, end = 0,0
start_position_within_gene = 0
end_position_within_gene = 0


if mode == 'region':
  start = 5000  #@param {type:"integer"}
  end = 10000  #@param {type:"integer"}
  if start == None or end == None or not isinstance(start,int) or not isinstance(end,int):
    start, end = 0,0
  Product_Size = abs(start-end)
  print(f'Target region: {cIndex}-{chromosomes[cIndex][0]}:{start}, {end}')
  if Product_Size <= 100:
    print(f'The specified Target region is too short. Are the start and end positions correct?')
    breake_execution = True
  if start > end:
    print(f'The start position is greater than the end position. Are the start and end positions correct?')
    breake_execution = True
elif mode == 'gene':
  #@markdown #### <br/>**Extract a subsequence of the gene [start and end nucleotides relative to the gene start]** <br/>For full gene length use: start_position_within_gene=0, end_position_within_gene=0 <br>[Only used in the gene mode]
  start_position_within_gene = 0 #@param {type:"integer"}
  end_position_within_gene = 0 #@param {type:"integer"}
  Product_Size = 0
else:
  #@markdown ### <br> **Options for promoter/terminator modes**

  #@markdown #### <br> ***Product size***. Number of nucleotide of the promoter or terminator region to extract.<br/> [Only in the promoter and terminator modes]
  Product_Size = 500 #@param {type:"integer"}
  if Product_Size == None or not isinstance(Product_Size,int):
    breake_execution = True

#@markdown ### <br> **General options**
#@markdown ##### ***Flexibility:*** how far away (nts) from the start/end coordinates can the designed primers bind. This option is for both sides unless fixedNear or fixedFar is selected.
flexibility = 200 #@param {type:"integer"}
if flexibility == None or not isinstance(flexibility,int):
  breake_execution = True
#@markdown FixedNear: the primer near start (promoter mode) or end (terminator mode) codon will be fixed. <br>
#@markdown FixedFar: the distant primer will be fixed.
fixedFar = False #@param {type:"boolean"}
fixedNear = False #@param {type:"boolean"}

if breake_execution:
  print('Error extracting target region...')
else:
  targetSeq,inStart,inEnd,prangemin,prangemax,FORCE_RIGHT_START,FORCE_LEFT_START = extractRegion(mode,cIndex,Product_Size,locus_tag=locus_tag,start=start,end=end, gpstart=start_position_within_gene, gpend=end_position_within_gene,fixedFar=fixedFar,fixedNear=fixedNear)


In [ ]:
#@title #Use a custom sequence [cut and paste]
#@markdown ###**Please input the target sequence [ATCG]**
target_sequence_id = '' #@param {type:"string"}
target_sequence = '' #@param {type:"string"}
target_sequence = target_sequence.upper()

with open('target.fasta','w') as f:
  f.write(f'>{target_sequence_id }\n{target_sequence}\n')

from Bio import SeqIO
targetSeq = SeqIO.read('target.fasta','fasta')

#@markdown ***Flexibility***: allows the program to design primers that bind a number of specified nucleotides away from the sequence ends. This option is used in both sides unless fixedLeft or fixedRight is selected.

flexibility = 200 #@param {type:"integer"}
#@markdown Select fixedLeft if you want the left primer to be anchored in the 5'-end of the target sequence. Select fixedRight if you want the right primer to be anchored in the 3'-end of the target sequence.
fixedLeft = False #@param {type:"boolean"}
fixedRight = False #@param {type:"boolean"}

FORCE_RIGHT_START = fixedRight
FORCE_LEFT_START = fixedLeft

if (not fixedLeft) and (not fixedRight):
  prangemin = len(targetSeq.seq) - 2*flexibility
elif fixedLeft and (not fixedRight):
  prangemin = len(targetSeq.seq) - flexibility
  inStart = 0
  inEnd = len(targetSeq.seq) - flexibility
elif (not fixedLeft) and fixedRight:
  prangemin = len(targetSeq.seq) - flexibility
  inStart = flexibility
  inEnd = len(targetSeq.seq) - 1
else:
  inStart = flexibility
  inEnd = len(targetSeq.seq) - flexibility
  prangemin = len(targetSeq.seq)

prangemax = len(targetSeq.seq)
print(f'Target product size (target.fasta): {len(targetSeq)}')
print(f'Flexible Product size range: {prangemin}-{prangemax}')
print(f'{targetSeq.seq}')

#***Evaluate the presence of restriction enzyme sites in the target sequence and add custom oligos in the 5'end of primers***

In [ ]:
#@title ##Evaluate the presence of REs sites in the target sequence
from Bio import Restriction

rb_supp = Restriction.RestrictionBatch(first=[], suppliers=['C','B','E','I','K','J','M',
'O','N','Q','S','R','V','Y','X'])

cutters = [r for r,s in rb_supp.search(targetSeq.seq).items() if len(s) > 0]
noncutters = [r for r,s in rb_supp.search(targetSeq.seq).items() if len(s) == 0]

#filter results by list
#@markdown ### **Add the Restriction enzyme names as a comma separated list.**
ER_list = 'EcoRI,BamHI,HindIII,XbaI' #@param {type:'string'}
ER_list = ER_list.replace(' ','')
ER_list = ER_list.split(',')

print(f'Analyzing sequence for restriction sites... {ER_list}')

for e in cutters:
  if str(e) in ER_list:
    print(f'X  --> The restriction enzyme {e} cuts the target sequence.')

for e in noncutters:
  if str(e) in ER_list:
    print(f'OK --> The restriction enzyme {e} does not cut the target sequence.')

print_all_noncutters = False #@param {type:"boolean"}
if print_all_noncutters or len(ER_list) == 0:
  print('\nAll noncutters RE:')
  for i in range(0,len(noncutters),10):
    for e in noncutters[i:i+10]:
      print(f'{str(e)}  ', end=(''))
    print()

In [ ]:
#@title ##Add 5' Oligonucleotides and restriction enzyme regcognition sites to the *fwd* and *rv* primers
#@markdown The forward (fwd) primer binds on the 5' end of the target sequence. <br> The reverse (rv) binds in the 3' end of the targetr sequence.


#@markdown 5'- [oligo\_5'-] [ER] [Primer Sequence] -3'<br>
#@markdown Input the oligonucleotides, or Restriction enzyme by name (eg. EcoRI, case matters)


oligo_5_prime_Fwd = 'AA' #@param {type:"string"}

if not validate_oligo(oligo_5_prime_Fwd) and oligo_5_prime_Fwd != '':
  print(f'Error: {oligo_5_prime_Fwd} is not a valid oligo sequence.')
  oligo_5_prime_Fwd = ''


enzyme_fwd = 'EcoRI' #@param {type:'string'}
if not enzyme_fwd == '':
  if enzyme_fwd in rb_supp:
    enzime_site_f = rb_supp.get(enzyme_fwd).site
    oligo_5_prime_Fwd2 = oligo_5_prime_Fwd + enzime_site_f
    print(f"Oligo 5'-Fwd: {oligo_5_prime_Fwd} {enzime_site_f} --> primer sequence'")
  elif validate_oligo(enzyme_fwd):
    oligo_5_prime_Fwd2 = oligo_5_prime_Fwd + enzyme_fwd
    print(f"Oligo 5'-Fwd: {oligo_5_prime_Fwd} {enzyme_fwd} --> primer sequence'")
  else:
    print(f'Error: {enzyme_fwd} not found in RE database')
else:
  oligo_5_prime_Fwd2 = oligo_5_prime_Fwd
  print(f"Fwd primer 5'- Oligosnucleotides were not added.")
#@markdown

oligo_5_prime_rv = 'AA' #@param {type:"string"}
if not validate_oligo(oligo_5_prime_rv) and oligo_5_prime_rv != '':
  print(f'Error: {oligo_5_prime_rv} is not a valid oligo sequence.')
  oligo_5_prime_rv = ''

enzyme_rv = 'EcoRI'  #@param {type:'string'}
if not enzyme_fwd == '':
  if enzyme_rv in rb_supp:
    enzime_site_r = rb_supp.get(enzyme_rv).site
    oligo_5_prime_rv2 = oligo_5_prime_rv + enzime_site_r
    print(f"Oligo 5'-Rv : {oligo_5_prime_rv} {enzime_site_r } --> primer sequence'")
  elif validate_oligo(enzyme_rv):
    oligo_5_prime_rv2 = oligo_5_prime_rv + enzyme_rv
    print(f"Oligo 5'-Rv : {oligo_5_prime_rv} {enzyme_rv} --> primer sequence'")
  else:
    print(f'Error: {enzyme_rv} not found in RE database')
else:
  oligo_5_prime_rv2 = oligo_5_prime_rv
  print(f"Rv primer 5'- Oligosnucleotides were not added.")

lfws2 =len(oligo_5_prime_Fwd2)
lrs2 = len(oligo_5_prime_rv2)
avg_oli=(lfws2+lrs2)/2

#***Run primer3 for primer design with default options***

In [ ]:
#@title ##Run Primer3 program
#@markdown ###For Primer3 options, see the [Primer 3 manual](https://primer3.org/manual.html).
#@markdown <br>

#**Input file options:**<br>SEQUENCE_ID=<br>SEQUENCE_TEMPLATE=<br>PRIMER_TASK=<br>PRIMER_EXPLAIN_FLAG=<<br>SEQUENCE_OVERHANG_LEFT=<br>SEQUENCE_OVERHANG_RIGHT=<br>PRIMER_PRODUCT_SIZE_RANGE=<br>SEQUENCE_FORCE_RIGHT_START=

#@markdown ### **Primer length options** (without 5'- oligonucleotides)
PRIMER_OPT_SIZE=18 #@param {type:"integer"}
PRIMER_MIN_SIZE=15 #@param {type:"integer"}
PRIMER_MAX_SIZE=25 #@param {type:"integer"}

#@markdown ###<br> **Tm options**
PRIMER_OPT_TM=60 #@param {type:"integer"}
PRIMER_MAX_TM=63 #@param {type:"integer"}
PRIMER_MIN_TM=57 #@param {type:"integer"}
PRIMER_PAIR_MAX_DIFF_TM = 3 #@param {type:"integer"}

#@markdown #### <br> To add custom options please upload a txt file with custom Primer3 configurations
Custom_Options = False #@param {type:"boolean"}
Custom_Options_file = '' #@param {type:'string'}

if Custom_Options:
  print('A custom configuration file will be used.')
  if os.path.exists(Custom_Options_file):
    configuration_file = Custom_Options_file
  else:
    print('Please upload your configuration file.')
    from google.colab import files
    uploaded = files.upload()
    if len(uploaded) > 0:
      configuration_file = uploaded.keys()[0]
else:
  configuration_file = 'input.txt'
  with open(configuration_file,'w') as f:
    f.write(f'''SEQUENCE_ID={cIndex}-{chromosomes[cIndex][0]}|{start}-{end}
SEQUENCE_TEMPLATE={targetSeq.seq}
PRIMER_TASK=generic
PRIMER_EXPLAIN_FLAG=1
PRIMER_PICK_ANYWAY=1
PRIMER_OPT_SIZE={int(PRIMER_OPT_SIZE)}
PRIMER_MIN_SIZE={int(PRIMER_MIN_SIZE)}
PRIMER_MAX_SIZE={int(PRIMER_MAX_SIZE)}
PRIMER_OPT_TM={PRIMER_OPT_TM}
PRIMER_MAX_TM={PRIMER_MAX_TM}
PRIMER_MIN_TM={PRIMER_MIN_TM}
PRIMER_PAIR_MAX_DIFF_TM={PRIMER_PAIR_MAX_DIFF_TM}\n''')
    #SEQUENCE_OVERHANG_LEFT / SEQUENCE_OVERHANG_RIGHT
    if len(oligo_5_prime_Fwd2) > 0:
      f.write(f'SEQUENCE_OVERHANG_LEFT={oligo_5_prime_Fwd2}\n')
    if len(oligo_5_prime_rv2) > 0:
      f.write(f'SEQUENCE_OVERHANG_RIGHT={oligo_5_prime_rv2}\n')
    if prangemin != prangemax:
      f.write(f'''PRIMER_PRODUCT_SIZE_RANGE={prangemin}-{prangemax}\n''')
    if FORCE_LEFT_START:
      f.write(f'''SEQUENCE_FORCE_LEFT_START={inStart}\n''')
    if FORCE_RIGHT_START:
      f.write(f'''SEQUENCE_FORCE_RIGHT_START={inEnd}\n''')
    if Custom_Options:
      with open(coptions,'r') as cop:
        for l in cop:
          f.write(l)
    f.write(f'=')

import os
output = 'primers.txt'
if os.path.exists(output):
  os.remove(output)

try:
  os.system(f'primer3_core < {configuration_file} > primers.txt')
  print(f'The primers were successfully designed: primers.txt')
except:
  print('Error designing primers')

In [ ]:
#@title ##Load the primers and generate the primer table.
#@markdown Reads the primer3 results and extracts the primer information
import pandas as pd

with open('primers.txt','r') as f:
  primers = f.read()

for l in primers.splitlines():
  if l.startswith('PRIMER_LEFT_NUM_RETURNED='):
    nLP = int(l.split('=')[1])
  elif l.startswith('PRIMER_RIGHT_NUM_RETURNED='):
    nRP = int(l.split('=')[1])

if (nLP > 0) and (nRP > 0):
  nprimers = min(nLP,nRP)
  #read primers
  primersDict = {}
  for i in range(nprimers):
    primerInfo = {}
    for l in primers.splitlines():
      if  l.startswith(f'PRIMER_PAIR_{i}_PENALTY'):
        primerInfo['PairPenalty'] = l.strip().split('=')[1]
      elif  l.startswith(f'PRIMER_LEFT_{i}_PENALTY'):
        primerInfo['fwdPenalty'] = l.strip().split('=')[1]
      elif  l.startswith(f'PRIMER_RIGHT_{i}_PENALTY'):
        primerInfo['rvPenalty'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_SEQUENCE'):
        primerInfo['fwdSeq'] = l.strip().split('=')[1]
        primerInfo['fwdLen'] = len(l.strip().split('=')[1])
      elif l.startswith(f'PRIMER_RIGHT_{i}_SEQUENCE'):
        primerInfo['rvSeq'] = l.strip().split('=')[1]
        primerInfo['rvLen'] = len(l.strip().split('=')[1])
      elif l.startswith(f'PRIMER_LEFT_{i}_TM'):
        primerInfo['fwdTM'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_TM'):
        primerInfo['rvTM'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_GC_PERCENT'):
        primerInfo['fwdGC'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_GC_PERCENT'):
        primerInfo['rvGC'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_PROBLEMS'):
        primerInfo['fwdProblems'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_PROBLEMS'):
        primerInfo['rvProblems'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_SELF_ANY_TH'):
        primerInfo['fwdSelfAny'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_SELF_ANY_TH'):
        primerInfo['rvSelfAny'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_SELF_END_TH'):
        primerInfo['fwdSelfEnd'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_SELF_END_TH'):
        primerInfo['rvSelfEnd'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_HAIRPIN_TH'):
        primerInfo['fwdHairpin'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_HAIRPIN_TH'):
        primerInfo['rvHairpin'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_LEFT_{i}_END_STABILITY'):
        primerInfo['fwdEndStability'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_RIGHT_{i}_END_STABILITY'):
        primerInfo['rvEndStability'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_PAIR_{i}_COMPL_ANY_TH'):
        primerInfo['complAny'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_PAIR_{i}_COMPL_END_TH'):
        primerInfo['complEnd'] = l.strip().split('=')[1]
      elif l.startswith(f'PRIMER_PAIR_{i}_PRODUCT_SIZE'):
        primerInfo['productSize'] = l.strip().split('=')[1]
      elif  l.startswith(f'PRIMER_PAIR_{i}_PRODUCT_TM'):
        primerInfo['productTM'] = l.strip().split('=')[1]
    primersDict[i] = primerInfo

primerTable = pd.DataFrame.from_dict(primersDict, orient='index', columns = ['PairPenalty',
'fwdPenalty','rvPenalty','fwdSeq','fwdLen','rvSeq','rvLen','fwdTM','rvTM','fwdGC','rvGC','fwdProblems','rvProblems',
'fwdSelfAny','rvSelfAny','fwdSelfEnd','rvSelfEnd','fwdHairpin','rvHairpin','fwdEndStability',
'rvEndStability','complAny','complEnd','productSize','productTM'])
# (names=['name','TM','Secuencia','Comentarios'])
primerTable.to_csv('primerTable.csv')

if len(primerTable) == 0:
  print('Primer3 could not find any primers with the provided options. Try relaxing Tm settings.')
primerTable

#***Blastn Virtual PCR***.

In [ ]:
#@title #Search for the primer sequences in the target genome
#@markdown ###This block searches for all the primer sets in the target genome.

with open('virtualPCR.csv','w') as f:
  f.write('SSeqID,SStart,SEnd,Fwd,FwdLen,#FwdFound,#FwdFoundWithMismatch,AlnLength,FwdSeq,SSeq,PIdent,Mismatch,Rv,RvLen,#RvFound,#RvFoundWithMismatch,AlnLength,RvSeq,SSeq,PIdent,Mismatch,ProductSize\n')
#@markdown ###<br> **Maximum difference in primer vs alignment length**
max_diff_aln_len = 3 #@param {type:"integer"}
#@markdown ####<br> Should 5'-oligos be removed from primers for virtual PCR?
remove_oligos = True #@param {type:'boolean'}

#primerTable
verify_first_primer_Set_only = True  #@param {type:"boolean"}

if verify_first_primer_Set_only:
  r = primerTable.iloc[0]
  i = 0
  q = 'primer_pair.tmp'
  with open(q,'w') as f:
    f.write(f">{i}_Fwd\n{str(r['fwdSeq'])}\n>{i}_rv\n{str(r['rvSeq'])}")
  print(f"Procesando primers {i}_Fwd:{str(r['fwdSeq'])} | {i}_rv:{str(r['rvSeq'])}")
  numProducts = primer_blast(q,oligo_5_prime_Fwd2,oligo_5_prime_rv2,gfile,i,max_diff_aln_len,remove_oligos)
else:
  for i,r in primerTable.iterrows():
    q = 'primer_pair.tmp'
    with open(q,'w') as f:
      f.write(f">{i}_Fwd\n{str(r['fwdSeq'])}\n>{i}_rv\n{str(r['rvSeq'])}")
    print(f"Procesando primers {i}_Fwd:{str(r['fwdSeq'])} | {i}_rv:{str(r['rvSeq'])}")
    numProducts = primer_blast(q,oligo_5_prime_Fwd2,oligo_5_prime_rv2,gfile,i,max_diff_aln_len,remove_oligos)

print(f'The results are in the virtualPCR.csv file.')
table = pd.read_csv('virtualPCR.csv')
print('The complete filtered blast table is in the FilteredPrimerHits_[primer pair].csv file.')
table

In [ ]:
#@title ###Search in the target genome custom designed primers
#@markdown ###VirtualPCR with custom primers
#@markdown [Requires a genome file to be downloaded or a local genome file]

#@markdown #### **Input the custom primers**
import pandas as pd
if os.path.exists('virtualPCR.csv'):
  os.remove('virtualPCR.csv')

with open('virtualPCR.csv','w') as f:
  f.write('SSeqID,SStart,SEnd,Fwd,FwdLen,#FwdFound,#FwdFoundWithMismatch,AlnLength,FwdSeq,SSeq,PIdent,Mismatch,Rv,RvLen,#RvFound,#RvFoundWithMismatch,AlnLength,RvSeq,SSeq,PIdent,Mismatch,ProductSize\n')

oligo_5_prime_Fwd_custom = '' #@param {type:"string"}
specific_sequence_Fwd = '' #@param {type:"string"}

oligo_5_prime_rv_custom = '' #@param {type:"string"}
specific_sequence_rv = '' #@param {type:"string"}

#@markdown ####<br> Should 5'-oligos be removed from primers for virtual PCR?
remove_oligos = True #@param {type:'boolean'}

#@markdown ###<br>Maximum difference in primer vs alignment length
max_diff_aln_len = 3 #@param {type:"integer"}

q = 'primer_pair.tmp'
with open(q,'w') as f:
  f.write(f">Custom_Fwd\n{oligo_5_prime_Fwd_custom}{specific_sequence_Fwd}\n>Custom_Rv\n{oligo_5_prime_rv_custom}{specific_sequence_rv}")

numProducts = primer_blast(q,oligo_5_prime_Fwd_custom,oligo_5_prime_rv_custom,gfile,'custom',max_diff_aln_len,remove_oligos)
table = pd.read_csv('virtualPCR.csv')
table
